In [6]:
#!pip install opencv-python matplotlib ipykernel
import cv2
import numpy as np

In [18]:
def load_video(path, decim = 2, cvtColor = None):
    frames = []
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if cvtColor:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        it=0
        while it < decim:
            rows, cols = frame.shape[0], frame.shape[1]
            frame = cv2.pyrDown(frame, dstsize=(cols // 2, rows // 2))
            it+=1
        
        frames.append(frame)
    return frames

def read_video(path, fps = 60, winname ="video"):
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        ret, frame = cap.read()
        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imshow(winname, gray)
        if cv2.waitKey(1000//fps) == ord('q'):
            break
        if not cv2.getWindowProperty(winname, cv2.WND_PROP_VISIBLE):  # when window is close get out of the loop
            break
    cap.release()
    cv2.destroyAllWindows()
    
def write_video(video, path, fps = 24):
    dsize = (video[0].shape[1],  video[0].shape[0])
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(path, fourcc, 30, dsize)
    for frame in video:
        out.write(frame)

#read_video('C:/Users/ac268552/Desktop/VI/Vision industrielle/video/veh2.mp4')
test = load_video('C:/Users/ac268552/Desktop/VI/Vision industrielle/video/veh2.mp4')
write_video(test,'C:/temp/output2.avi')    
print(len(test))
print(test[0].shape)

2913
(270, 480, 3)
